In [1]:
import os
import logging
import pandas as pd
import matplotlib.pyplot as plt

# Import classes and functions from the fits_metadata_extractor package
from fits_metadata_extractor.processor import FITSProcessor
from fits_metadata_extractor.utils import save_metadata_to_csv, load_metadata_from_csv
from fits_metadata_extractor.search import search_fits_by_point, search_fits_by_region
from fits_metadata_extractor.plotter import plot_moc_and_polygon_from_dataset_notebook
from fits_metadata_extractor.logger import setup_logging


In [3]:
# Initialize and configure logging
logger = setup_logging()


In [4]:
# Define the directory containing FITS files
fits_directory = "fits_collection"  # <-- CHANGE THIS PATH TO YOUR TEST FITS FILES DIRECTORY

# Define the output CSV file path
output_csv = 'test_metadata.csv'

# Define the directory to save plots
plot_output_dir = 'test_plots'

if not os.path.isdir(fits_directory):
    logger.error(f"The specified FITS directory does not exist: {fits_directory}")
else:
    logger.info(f"FITS directory found: {fits_directory}")


2024-11-08 10:25:49,959 - INFO - FITS directory found: fits_collection
2024-11-08 10:25:49,959 - INFO - FITS directory found: fits_collection


In [5]:
# Instantiate FITSProcessor with desired number of workers
processor = FITSProcessor(max_workers=5)

# Process the FITS directory
metadata_df = processor.process_fits_directory_parallel(fits_directory)


2024-11-08 10:25:52,163 - INFO - Processing file: fits_collection/G327.617-0.364_I4.fits
2024-11-08 10:25:52,163 - INFO - Processing file: fits_collection/SN1987A_cut_35_smHB.fits
2024-11-08 10:25:52,163 - INFO - Processing file: fits_collection/G327.617-0.364_I4.fits
2024-11-08 10:25:52,163 - INFO - Processing file: fits_collection/G351.702+0.672_atlasgal.fits
2024-11-08 10:25:52,163 - INFO - Processing file: fits_collection/SN1987A_cut_35_smHB.fits
2024-11-08 10:25:52,164 - INFO - Processing file: fits_collection/5GHz_n_f.fits
2024-11-08 10:25:52,163 - INFO - Processing file: fits_collection/G351.702+0.672_atlasgal.fits
2024-11-08 10:25:52,164 - INFO - Processing file: fits_collection/G351.632-0.459_atlasgal.fits
2024-11-08 10:25:52,164 - INFO - Processing file: fits_collection/5GHz_n_f.fits
2024-11-08 10:25:52,164 - INFO - Processing file: fits_collection/G351.632-0.459_atlasgal.fits
2024-11-08 10:25:52,169 - INFO - Inferred object name 'G327.617-0.364' for file 'fits_collection/G32

In [6]:
# Show metadata_df content 

logger.info(f"Metadata DataFrame describe: {metadata_df.head()}")


2024-11-08 10:26:09,877 - INFO - Metadata DataFrame describe:                       FITS_File  \
0  G351.702+0.672_atlasgal.fits   
1  G351.632-0.459_atlasgal.fits   
2      UGC_09618_S_2MASS_H.fits   
3        G327.617-0.364_I4.fits   
4      SN1987A_cut_35_smHB.fits   

                                             Polygon  \
0  POLYGON ((351.8408317015 0.6008334535, 351.552...   
1  POLYGON ((351.7458316825 -0.5191667704999999, ...   
2  POLYGON ((224.30832666152753 24.32279635279270...   
3  POLYGON ((327.7141602095 -0.4191667505, 327.51...   
4  POLYGON ((83.98078838261291 -69.31003927205, 8...   

                                                 MOC  \
0  9/2717291 2717294 10/10869158-10869159 1086916...   
1  9/2716914-2716915 10/10867613 10867615 1086765...   
2  9/575820 575822-575823 575845 575856 575858-57...   
3  9/2676076 10/10704293-10704295 10704300-107043...   
4                       9/2118062 10/8472237 8472239   

                                      Polygon_Coords 

In [7]:
# Save the extracted metadata to CSV
save_metadata_to_csv(metadata_df, output_csv)


2024-11-08 10:26:11,683 - INFO - Metadata successfully saved to test_metadata.csv.
2024-11-08 10:26:11,683 - INFO - Metadata successfully saved to test_metadata.csv.


In [7]:
# Load the metadata from CSV
metadata_loaded_df = load_metadata_from_csv(output_csv)

2024-11-04 09:40:28,844 - INFO - Metadata loaded from test_metadata.csv.


In [9]:
# Test plotting using plot_moc_and_polygon_from_dataset_notebook
logger.info("Testing plotting of MOCs and polygons.")

try:
    plot_moc_and_polygon_from_dataset_notebook(
        metadata_df=metadata_loaded_df,
        input_dir=fits_directory,
        output_dir=plot_output_dir,
        max_plots=16  # Limit to 5 plots for testing
    )
    logger.info("Finished plotting MOCs and polygons.")
except Exception as e:
    logger.error(f"Error occurred during plotting: {e}")


2024-11-04 09:40:48,254 - INFO - Testing plotting of MOCs and polygons.
2024-11-04 09:40:48,255 - INFO - Plots will be saved to: test_plots
2024-11-04 09:40:48,255 - INFO - No specific selection criteria provided. All FITS files will be plotted.
2024-11-04 09:40:48,256 - INFO - Generating 16 plots.


Generating Plots:   0%|          | 0/16 [00:00<?, ?it/s]

2024-11-04 09:40:48,369 - INFO - Calling _reproject_full in non-dask mode
2024-11-04 09:40:48,370 - ERROR - Failed to reproject or plot FITS data: Number of dimensions in input and output WCS should match
2024-11-04 09:40:48,921 - INFO - Plot saved as 5GHz_n_f.png
2024-11-04 09:40:48,921 - INFO - Plot saved: test_plots/5GHz_n_f.png
2024-11-04 09:40:48,937 - INFO - Calling _reproject_full in non-dask mode
2024-11-04 09:40:50,080 - INFO - Plot saved as G351.702+0.672_atlasgal.png
2024-11-04 09:40:50,081 - INFO - Plot saved: test_plots/G351.702+0.672_atlasgal.png
2024-11-04 09:40:50,116 - INFO - Calling _reproject_full in non-dask mode
2024-11-04 09:40:51,175 - INFO - Plot saved as UGC_09618_S_2MASS_H.png
2024-11-04 09:40:51,176 - INFO - Plot saved: test_plots/UGC_09618_S_2MASS_H.png
2024-11-04 09:40:51,196 - INFO - Calling _reproject_full in non-dask mode
2024-11-04 09:40:52,253 - INFO - Plot saved as UGC_09618_2MASS_H.png
2024-11-04 09:40:52,254 - INFO - Plot saved: test_plots/UGC_09618

In [10]:
# Define a test point
test_point = {
    'ra': 150.0,  # Right Ascension in degrees
    'dec': 2.2    # Declination in degrees
}

logger.info(f"Testing search by point: RA={test_point['ra']}, Dec={test_point['dec']}")

# Perform the search
matching_fits_by_point = search_fits_by_point(metadata_loaded_df, test_point['ra'], test_point['dec'])

# Display results
if not matching_fits_by_point.empty:
    logger.info(f"FITS files containing the point (RA={test_point['ra']}, Dec={test_point['dec']}):")
    display(matching_fits_by_point[['FITS_File', 'Resolved_Object']])
else:
    logger.info("No FITS files found containing the specified point.")


2024-11-04 09:42:26,852 - INFO - Testing search by point: RA=150.0, Dec=2.2
2024-11-04 09:42:26,857 - INFO - FITS files containing the point (RA=150.0, Dec=2.2):


,FITS_File,Resolved_Object
12,id12_GAL-Survey-GC.fpsf.fits,Unknown


In [11]:
# Define a test circular region
test_circle = {
    'type': 'circle',
    'center': (150.0, 2.2),  # Center RA and Dec in degrees
    'radius': 1.0             # Radius in degrees
}

logger.info(f"Testing search by circular region: Center RA={test_circle['center'][0]}, Dec={test_circle['center'][1]}, Radius={test_circle['radius']} degrees")

# Perform the search
matching_fits_by_circle = search_fits_by_region(metadata_loaded_df, test_circle)

# Display results
if not matching_fits_by_circle.empty:
    logger.info(f"FITS files intersecting the circular region (RA={test_circle['center'][0]}, Dec={test_circle['center'][1]}, Radius={test_circle['radius']} degrees):")
    display(matching_fits_by_circle[['FITS_File', 'Resolved_Object']])
else:
    logger.info("No FITS files found intersecting the specified circular region.")


2024-11-04 09:42:30,602 - INFO - Testing search by circular region: Center RA=150.0, Dec=2.2, Radius=1.0 degrees
2024-11-04 09:42:30,612 - INFO - FITS files intersecting the circular region (RA=150.0, Dec=2.2, Radius=1.0 degrees):


,FITS_File,Resolved_Object
4,2013.1.01292.S_SB_X4eb_GB_X4ec_MB_X4ed_2-38011...,Unknown
12,id12_GAL-Survey-GC.fpsf.fits,Unknown
14,2013.1.00034.S_SB_X5_GB_X6_MB_X7_midz_cell10_2...,Unknown


In [12]:
# Define a test polygonal region
test_polygon = {
    'type': 'polygon',
    'coordinates': [
        (149.0, 1.0),
        (151.0, 1.0),
        (151.0, 3.0),
        (149.0, 3.0)
    ]
}

logger.info(f"Testing search by polygonal region with coordinates: {test_polygon['coordinates']}")

# Perform the search
matching_fits_by_polygon = search_fits_by_region(metadata_loaded_df, test_polygon)

# Display results
if not matching_fits_by_polygon.empty:
    logger.info("FITS files intersecting the polygonal region:")
    display(matching_fits_by_polygon[['FITS_File', 'Resolved_Object']])
else:
    logger.info("No FITS files found intersecting the specified polygonal region.")


2024-11-04 09:42:33,604 - INFO - Testing search by polygonal region with coordinates: [(149.0, 1.0), (151.0, 1.0), (151.0, 3.0), (149.0, 3.0)]
2024-11-04 09:42:33,609 - INFO - FITS files intersecting the polygonal region:


,FITS_File,Resolved_Object
4,2013.1.01292.S_SB_X4eb_GB_X4ec_MB_X4ed_2-38011...,Unknown
12,id12_GAL-Survey-GC.fpsf.fits,Unknown
14,2013.1.00034.S_SB_X5_GB_X6_MB_X7_midz_cell10_2...,Unknown
